In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from PIL import Image, ImageOps


transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# Create datasets for training & validation, download if necessary
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

# Create data loaders for our datasets; shuffle for training, not for validation
training_loader = torch.utils.data.DataLoader(training_set, batch_size=1, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=4, shuffle=False)

# Class labels
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

# Report split sizes
print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(validation_set)))

Training set has 60000 instances
Validation set has 10000 instances


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Helper function for inline image display
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(training_loader)
images, labels = next(dataiter)

# Create a grid from the images and show them
# img_grid = torchvision.utils.make_grid(images)
# matplotlib_imshow(img_grid, one_channel=True)
# print('  '.join(classes[labels[j]] for j in range(4)))

In [ ]:
def resize(img):
    im = transforms.functional.to_pil_image(img)
    im = im.resize((32, 32), Image.BILINEAR)
    tensor_im = transforms.functional.to_tensor(im)
    if tensor_im.shape[0] == 1:  # If grayscale, convert to RGB
        tensor_im = torch.cat([tensor_im] * 3)
    return tensor_im

In [ ]:
import os
import requests
from pathlib import Path
import pandas as pd
from torch.utils.data import Dataset
from typing import Tuple
import torch
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, RandomResizedCrop


In [ ]:
class TaskDataset(Dataset):
    def __init__(self, transform=None):

        self.ids = []
        self.imgs = []
        self.labels = []

        self.transform = transform

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        if not self.transform is None:
            img = self.transform(img)
        label = self.labels[index]
        return id_, img, label

    def __len__(self):
        return len(self.ids)

In [ ]:
def get_path_to_img_dir():
  img_dir = os.path.join(os.getcwd(), 'resized_images1')
  if not os.path.isdir(img_dir):
      os.mkdir(img_dir)
  return img_dir

In [ ]:
for idx, (X, y) in enumerate(training_loader):
  img = resize(X[0])

  im = transforms.functional.to_pil_image(img)
  im.save(f'{get_path_to_img_dir()}/{idx}.png', format="PNG")

In [ ]:
dataset = torch.load("./ModelStealingPub.pt")

In [ ]:
import random

In [ ]:
representatives = [6842, 10477, 8845, 823, 7902, 25, 5744, 2257, 1396, 2745, 4739, 12647, 4218, 2443, 5576, 6550, 7298, 8989, 7251, 358, 1286, 7849, 2377, 9003, 28, 24, 4569, 11657, 9410, 9139, 9229, 10588, 5296, 9901, 11839, 6483, 11200, 3483, 7170, 12956, 10667, 7697, 6666, 8550, 5135, 7776, 8301, 4021, 911, 1171, 1371, 11063, 791, 8854, 858, 1958, 3969, 4179, 8759, 1611, 6159, 12966, 5021, 9902, 4422, 8595, 4420, 4456, 11009, 1357, 3337, 7906, 685, 6473, 3058, 8183, 10522, 1981, 12758, 12332, 12505, 2578, 1845, 6477, 6772, 9613, 5693, 2299, 5508, 153, 3144, 8127, 12333, 9677, 5012, 221, 11622, 789, 6854]
for i in random.choices(range(len(dataset)), k=150):
    if i not in representatives:
        representatives.append(i)


In [ ]:
def model_stealing():
    SERVER_URL = "http://34.71.138.79:9090"
    ENDPOINT = "/modelstealing"
    URL = SERVER_URL + ENDPOINT

    TEAM_TOKEN = "DdE56eC1EvRwWcuy"

    for image in dataset:
        id_, img, _ = image
        img = img.convert("RGB")
        img.save(f"output/{id_}.png")

    output = []

    for idx, file_name in enumerate(os.listdir('output')):
        if not file_name.endswith('.png'):
            continue
        if idx % 100 == 0:
            print(f'output Progress: {idx} in {len(dataset)}')
        with open(os.path.join('output', file_name), "rb") as img_file:
            response = requests.get(
                URL, files={"file": img_file}, headers={"token": TEAM_TOKEN}
            )

        if response.status_code == 200:
            output.append({"filename": file_name, "embeddings": response.json()["representation"]})

    # for idx, file_name in enumerate(os.listdir('aug')):
    #     if not file_name.endswith('.png'):
    #         continue
    #     if idx % 100 == 0:
    #         print(f'augmentation Progress: {idx} in {len(os.listdir("aug"))}')
    #     with open(os.path.join('aug', file_name), "rb") as img_file:
    #         response = requests.get(
    #             URL, files={"file": img_file}, headers={"token": TEAM_TOKEN}
    #         )

    #     if response.status_code == 200:
    #         output.append({"filename": file_name, "embeddings": response.json()["representation"]})


    df = pd.DataFrame(output, columns=["filename", "embeddings"])
    df.to_csv("output/embeddings.csv", index=False)

In [ ]:
def generate_augmentation(rotate=False, crop=False, color_jitter=False):
    transformations = []
    if rotate:
        transformations.append(transforms.RandomHorizontalFlip(p=1))
    if crop:
        transformations.append(transforms.RandomCrop(size=(32, 32), padding=4))
    if color_jitter:
        transformations.append(transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5))

    transform = Compose(transformations)

    # Directory where augmented images will be saved
    save_dir = 'aug/'
    os.makedirs(save_dir, exist_ok=True)

    for image in dataset:
        # Apply transformations
        id, img, _ = image
        for i in range(3):
            transformed_image = transform(image)

            # Save the transformed image
            save_path = os.path.join(save_dir, f"{id}_{i}.png")
            transformed_image.save(save_path)

In [ ]:
generate_augmentation(rotate=True, crop=True, color_jitter=True)

In [ ]:
res = model_stealing()

output Progress: 0 in 13000
output Progress: 100 in 13000
output Progress: 200 in 13000
output Progress: 300 in 13000
output Progress: 400 in 13000
output Progress: 500 in 13000
output Progress: 600 in 13000
output Progress: 700 in 13000
output Progress: 800 in 13000
output Progress: 900 in 13000
output Progress: 1000 in 13000
output Progress: 1100 in 13000
output Progress: 1200 in 13000
output Progress: 1300 in 13000
output Progress: 1400 in 13000
output Progress: 1500 in 13000
output Progress: 1600 in 13000
output Progress: 1700 in 13000
output Progress: 1800 in 13000
output Progress: 1900 in 13000
output Progress: 2000 in 13000
output Progress: 2100 in 13000
output Progress: 2200 in 13000
output Progress: 2300 in 13000
output Progress: 2400 in 13000
output Progress: 2500 in 13000
output Progress: 2600 in 13000
output Progress: 2700 in 13000
output Progress: 2800 in 13000
output Progress: 2900 in 13000
output Progress: 3000 in 13000
output Progress: 3100 in 13000
output Progress: 320

In [ ]:
import shutil

In [ ]:
outputs = ['output', 'aug']
i = 0
for output_dir in outputs:
  for filename in os.listdir(output_dir):
    if not filename.endswith('.png') and not filename.endswith('.csv'):
      continue
    if filename == 'embeddings.csv':
      final_filename = f'embeddings{i}.csv'
      i += 1
    else:
      final_filename = filename
    shutil.copy(os.path.join(output_dir, filename), os.path.join('final_output', final_filename))

In [ ]:
import pandas as pd

# List of file paths of CSV files to
dfs = []
for filename in os.listdir('final_output'):
  if not filename.endswith('.csv'):
    continue
  df = pd.read_csv(os.path.join('final_output', filename))
  dfs.append(df)

# Concatenate DataFrames into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Write the merged DataFrame to a new CSV file
merged_df.to_csv("final_embeddings.csv", index=False)

In [ ]:
!zip -r zipped_all.zip ./output/*

Streaming output truncated to the last 5000 lines.
  adding: output/268786.png (stored 0%)
  adding: output/268819.png (stored 0%)
  adding: output/268835.png (stored 0%)
  adding: output/268851.png (stored 0%)
  adding: output/268852.png (stored 0%)
  adding: output/268916.png (stored 0%)
  adding: output/268919.png (stored 0%)
  adding: output/268957.png (stored 0%)
  adding: output/268961.png (stored 0%)
  adding: output/268965.png (stored 0%)
  adding: output/268979.png (stored 0%)
  adding: output/269038.png (stored 0%)
  adding: output/269060.png (stored 0%)
  adding: output/269061.png (stored 0%)
  adding: output/269062.png (stored 0%)
  adding: output/269149.png (stored 0%)
  adding: output/269151.png (stored 0%)
  adding: output/269157.png (stored 0%)
  adding: output/269196.png (stored 0%)
  adding: output/269214.png (stored 0%)
  adding: output/269218.png (stored 0%)
  adding: output/269229.png (stored 0%)
  adding: output/269273.png (stored 0%)
  adding: output/269288.png (

# RESET

In [ ]:
def model_stealing_reset():
    SERVER_URL = "http://34.71.138.79:9090"
    ENDPOINT = "/modelstealing/reset"
    URL = SERVER_URL + ENDPOINT

    TEAM_TOKEN = "DdE56eC1EvRwWcuy"

    response = requests.post(
        URL, headers={"token": TEAM_TOKEN}
    )

    if response.status_code == 200:
        print("Endpoint rested successfully")
    else:
        raise Exception(f"Request failed. Status code: {response.status_code}, content: {response.content}")

In [ ]:
model_stealing_reset()

Endpoint rested successfully
